### pull down known DGA domains from: 

http://osint.bambenekconsulting.com/feeds/dga-feed.txt

In [ ]:
# import urllib
# target_url = 'https://ransomwaretracker.abuse.ch/downloads/RW_URLBL.txt'
# txt = urllib.urlopen(target_url).read()

import urllib.request
with urllib.request.urlopen("https://ransomwaretracker.abuse.ch/downloads/RW_URLBL.txt") as url:
    s = url.read()

#features 
full_grab = str(s).split('\\n') 
dga_list = str(s).split('\\n')[7:]
timestamp = str(s).split('\\n')[2]
name_of_list = str(s).split('\\n')[1]

In [ ]:
#create dataframe of dga_list
import pandas as pd
dga_list = pd.DataFrame(dga_list)
dga_list.columns = ['url']
#load only first 10,000 lines
dga_list = dga_list[:10000]

In [ ]:
#get today's date
import datetime
now = datetime.datetime.now().strftime("%Y-%m-%d")
print(now)
#dga_list.to_csv()
import pandas as pd
dga_list = pd.DataFrame(dga_list)
dga_list.to_csv('dga_domains_' + now + '.txt', index=False)

In [ ]:
#domain report 
import requests
import json
from pandas.io.json import json_normalize
import time
start = time.time()
print(start)

url = 'https://www.virustotal.com/vtapi/v2/domain/report'
#domain = ['www.coropeppinumereu.it']
f = open('dga_domains_' + now + '.txt', 'r')
domains = f.readlines()
domains = [d.strip() for d in domains]
domains = [i.strip('http://') for i in domains]
sep = '/'
domains = [s.split(sep, 1)[0] for s in domains]
domains.pop(0)


for value in domains:
    params = {'domain': value, 'apikey': ''}
    headers = {
      "Accept-Encoding": "gzip, deflate",
      "User-Agent" : "gzip,  My python example client or username"
    }
    response = requests.get(url, params=params, headers=headers).json()
    data = response
    json_normalize(data)
    
    
    with open('dga_domains_' + now + '.json', 'a') as outfile:
        json.dump(data, outfile, ensure_ascii=True)
        print(data)
        outfile.write('\n')

end = time.time()
print(end)
print(end - start)

In [ ]:
import json    

domain_data = []
with open('dga_domains_' + now + '.json') as f:
    for line in f:
        domain_data.append(json.loads(line))

In [ ]:
import json
from pandas.io.json import json_normalize
dga_report = json_normalize(domain_data)
dga_report.head()

#write to pickle file 
dga_report.to_pickle('dga_pickle_' + now + '.pkl')

In [ ]:
#url report
import requests
import pandas as pd
import time
start = time.time()
print(start)

url = 'https://www.virustotal.com/vtapi/v2/url/report'
headers = {
  "Accept-Encoding": "gzip, deflate",
  "User-Agent" : "gzip,  My python example client or username"
}

# read domains from file and pass them to DomainScanner and DomainReportReader
with open('dga_domains_' + now + '.txt', 'r') as infile:  # keeping the file open because it shouldnt
                                          # be opened/modified during reading anyway
    for value in infile:
        value = value.strip('\n')
        print(value)
        try:
            for value in infile:
                params = {"apikey": '', "resource": value}
                response =requests.get(url, params=params, headers=headers).json()
                json_response = response
                import json
                with open('url_report_' + now + '.json', 'a') as outfile:
                    json.dump(json_response, outfile)
                    outfile.write('\n')


        except Exception as err:  # keeping it
            print('Encountered an error but scanning will continue.', err)
            pass

end = time.time()
print(end)
print(end - start)

In [ ]:
import json    

url_report = []
with open('url_report_' + now + '.json') as f:
    for line in f:
        url_report.append(json.loads(line))

In [ ]:
import json
from pandas.io.json import json_normalize
dga_url_report = json_normalize(url_report)
dga_url_report.head()

#write to pickle file 
dga_url_report.to_pickle('url_report_' + now + '.pkl')

In [ ]:
#url parsing 
#parse url for features

#query
dga_list = dga_list['url'].replace('(^http\:\/\/)', '', regex=True, inplace=False)
dga_list = pd.DataFrame(dga_list)
dga_list_query = dga_list["url"].str.split("/", expand = True).rename(columns = lambda x: "query"+str(x+1))
dga_list = dga_list.join(dga_list_query)


#entropy
import math


def entropy(string):
        "Calculates the Shannon entropy of a string"

        # get probability of chars in string
        prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

        # calculate the entropy
        entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

        return entropy

#search for malicious file extension
#dga_list[dga_list['url'].str.contains("ball")]
searchfor = ['\.gz','\.vbs','\.vbe','\.7z','\.exe','\.js','\.jar','\.rar','\.ace','\.scr','\.bat','\.arj','\.lnk','\.mht','mht','wsf','js','fakertf','\.wsf','\.xz','Java Archive',
'\.r00','\.bz','\.uue','\.chm','\.pif','\.pdf','\.cab','\.pub','\.jse','\.vbs','vbs','\.dmg','\.sfx','\.py','\.bat','\.tmp','\.vbs','\.cmd','\.vbe','\.jse','\.lnk','\.PS1',
             '\.PS1XML','\.PS2','\.PS2XML','\.PSC1','\.PSC2','\.inf','\.msi','\.msp','\.gadget','\.zip']
pattern = '|'.join(searchfor)
dga_list['mal_ext'] = dga_list['url'].str.contains(pattern)
dga_list['file_ext'] = pd.DataFrame(dga_list['url'].str.findall(pattern))



#ip address
ippattern = (r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
dga_list['IP'] = dga_list['url'].str.contains(ippattern)

#tld
tldpattern = (r"\.\w+\/")
dga_list['tld'] = dga_list['url'].str.findall(tldpattern).astype(str)
dga_list['tld'] = dga_list['tld'].replace('[\[|\]|\/|\.|\']', '', regex=True, inplace=False)

#benign extensions
safeext = ['\.gif','\.jpg','\.jpeg','\.xml','\.csv','\.png','\.p7s','\.diff','\.mp4','\.wav','\.dat','\.txt','\.tif','\.mp3','\.tsv', 'JPEG graphics file', 
'Graphics interchange format file', 'Tagged Image File Format', 'MPEG-1 Audio Layer (MP3) audio file','noextension','Portable Network Graphics file',
'csv','txt','xml','\.ics','Java bytecode file','fp', 'fp_senders_mailfrom', 'fp_sender_from']
pattern2 = '|'.join(safeext)
dga_list['safe_ext'] = dga_list['url'].str.contains(pattern2, regex=True)

dga_list['safe_file_ext'] = pd.DataFrame(dga_list['url'].str.findall(pattern2))
dga_list



